In [1]:
!pip install -q -U pyarrow==14.0.1
!pip install -q -U fsspec==2023.10.0
!pip install  accelerate
!pip install bitsandbytes
!pip install -q -U peft==0.8.2
!pip install -q -U trl==0.7.10
!pip install --upgrade tensorflow protobuf
!pip install -q -U datasets==2.17.0
!pip install --upgrade trl
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 9.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2023.10.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━

In [5]:
from huggingface_hub import login

login()

**Loading the Model and Tokenizer**

In [6]:
import accelerate
import bitsandbytes as bnb
print("Accelerate version:", accelerate.__version__)
print("BitsAndBytes version:", bnb.__version__)

Accelerate version: 1.2.1
BitsAndBytes version: 0.45.0


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import transformers
import torch


# Set the qunatization config
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

#Load the model and Tokenizer
model_id = "google/gemma-2-2b-it"

# Disable automatic device placement by accelerate
# This is done by setting `device_map="auto"` which disables accelerate's device placement logic
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    quantization_config=quantization_config,
)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

**Dataset Preparation**

In [8]:
from datasets import load_dataset

dataset = load_dataset("ruslanmv/ai-medical-chatbot")
dataset

README.md:   0%|          | 0.00/863 [00:00<?, ?B/s]

dialogues.parquet:   0%|          | 0.00/142M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Description', 'Patient', 'Doctor'],
        num_rows: 256916
    })
})

In [9]:
# Convert HF dataset to pandas Dataframe

df = dataset["train"].to_pandas()
df.sample(10)

,Description,Patient,Doctor
117793,What causes swelling of lower part of legs wit...,"Hi Doc, i am on Prolomet XL 50 after undergoin...",Hi I did review your concern.Prolomet should n...
209104,"Baby was given DTP and Jaundice injections, sh...","I have a girl baby she is 4months old, First 2...","yes, u should give remaining 2 injection of br..."
26548,"Q. Every time I eat spicy food, I poop blood. ...","Hi doctor, I am a 26 year old male. I am 5 fee...",Hello. I have gone through your information an...
22190,Q. How to treat itchy bumps on my vagina?,"Hello doctor, I am a 25 year old unmarried fem...",Hi. It appears to be a contact dermatitis supe...
234453,Can niaspan cause oral lichen planus?,Can Niaspan cause oral lichen planus? I have a...,"HiNo, Niacin is a vitamin used in the treatmen..."
33751,Is Sertraline the right medicine for anxiety?,I have just been prescribed Sertraline HCL 25 ...,Hello and Welcome to ‘Ask A Doctor’ service.I ...
59509,How can one avoid the practice of masturbation?,I am 21 years old. I started masturbating when...,HelloThanks for query .Based on the facts that...
18561,Q. I am unable to swallow due to a painful lum...,"Hi doctor, I have a painful lump under my tons...",Hi. I have seen the picture (attachment remove...
208907,What should i do for SGPT is raised to 149 ?,my SGPT IS RAISED149,Elevated SGPT indicates a problem with liver f...
236621,Could taking Atacand plus result in tiredness ...,I take a combination therapy antihypertensive ...,"Hello,Atacand plus contains an ARB and a diure..."


In [10]:
def generate_prompt(data_point):
    # Generate prompt
    prefix_text = 'Below is an instruction that describes a task. Write a response that ' \
                  'appropriately completes the request.\n\n'

    # Samples with additional context info
    if data_point['Patient']:
        text = f"""<start_of_turn>user {prefix_text} {data_point["Description"]} here are the inputs {data_point["Patient"]} <end_of_turn>\n<start_of_turn>model{data_point["Doctor"]} <end_of_turn>"""
    # Without additional context info
    else:
        text = f"""<start_of_turn>user {prefix_text} {data_point["Description"]} <end_of_turn>\n<start_of_turn>model{data_point["Doctor"]} <end_of_turn>"""
    return text

# Add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in dataset["train"]]
dataset = dataset["train"].add_column("prompt", text_column)
dataset

Dataset({
    features: ['Description', 'Patient', 'Doctor', 'prompt'],
    num_rows: 256916
})

In [11]:
print(dataset[0]['prompt'])

<start_of_turn>user Below is an instruction that describes a task. Write a response that appropriately completes the request.

 Q. What does abutment of the nerve root mean? here are the inputs Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for annular bulging and tear? <end_of_turn>
<start_of_turn>modelHi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online --> <end_of_turn>


In [12]:
dataset = dataset.shuffle(seed=1234)  # Shuffle dataset
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/256916 [00:00<?, ? examples/s]

**Train-Test Split**

In [13]:
dataset = dataset.train_test_split(test_size=0.1)
train_data = dataset["train"]
test_data = dataset["test"]

print(train_data)
print(test_data)

Dataset({
    features: ['Description', 'Patient', 'Doctor', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 231224
})
Dataset({
    features: ['Description', 'Patient', 'Doctor', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 25692
})


**Low-Rank Adaptation for Large Language Models (LoRA)**

In [14]:
# Automated selection of target modules
import bitsandbytes as bnb

def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)


modules = find_all_linear_names(model)
print(modules)

['q_proj', 'v_proj', 'k_proj', 'o_proj', 'up_proj', 'gate_proj', 'down_proj']


**Modules**

In [15]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

print(model)

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear4bit(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm):

In [16]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # Causal Language Modeling (e.g., autoregressive models like GPT)
)

model = get_peft_model(model, lora_config)

**Train Model**

Push the trained model to HF

In [17]:
# Number of trainable parameters
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | Total: {total} | Percentage: {100 * trainable / total:.4f}%")

Trainable: 83066880 | Total: 2697408768 | Percentage: 3.0795%


In [19]:
import transformers
from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
torch.cuda.empty_cache()

# Define a formatting function to extract the 'prompt' field from the dataset
def formatting_func(example):
    return example['prompt']

# Preprocess the datasets using the tokenizer with max_length
train_data = train_data.map(lambda samples: tokenizer(samples["prompt"], padding='max_length', truncation=True, max_length=512), batched=True)
test_data = test_data.map(lambda samples: tokenizer(samples["prompt"], padding='max_length', truncation=True, max_length=512), batched=True)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    peft_config=lora_config,
    formatting_func=formatting_func,  # Use formatting_func instead of dataset_text_field
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=3,  # Changed from 0.03 to 3
        max_steps=100,
        num_train_epochs=2,
        learning_rate=2e-4,
        logging_steps=20,
        output_dir='outputs',
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        report_to="none",
        # Removed preprocessing_kwargs
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False
torch.cuda.empty_cache()
trainer.train()

Map:   0%|          | 0/231224 [00:00<?, ? examples/s]

Map:   0%|          | 0/25692 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:359: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a valid formatting function. Therefore `formatting_func` will be ignored. Either remove the `formatting_func` or pass a dataset that is not already processed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is

Step,Training Loss
20,2.973900
40,2.536900
60,2.458000
80,2.432900
100,2.451300


TrainOutput(global_step=100, training_loss=2.570606231689453, metrics={'train_runtime': 1195.2529, 'train_samples_per_second': 0.335, 'train_steps_per_second': 0.084, 'total_flos': 2589800162918400.0, 'train_loss': 2.570606231689453, 'epoch': 0.001729924229318756})

In [20]:
new_model = "gemma-2b-instruct-ft-ai-medical"

trainer.model.save_pretrained(new_model)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model locally
# save_adapter=True, save_config=True
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/11andrea2233/gemma-2b-instruct-ft-ai-medical/commit/8a93f00f0ffa2887642ae8ddc8e5f61f74224145', commit_message='Upload tokenizer', commit_description='', oid='8a93f00f0ffa2887642ae8ddc8e5f61f74224145', pr_url=None, repo_url=RepoUrl('https://huggingface.co/11andrea2233/gemma-2b-instruct-ft-ai-medical', endpoint='https://huggingface.co', repo_type='model', repo_id='11andrea2233/gemma-2b-instruct-ft-ai-medical'), pr_revision=None, pr_num=None)

**Evaluation Metrics**

In [24]:
# Load/define base (non-finetuned) vs finetuned models for comparison

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from math import exp
import bitsandbytes as bnb  # Ensure bitsandbytes is installed if using quantization
from peft import PeftModel  # If using PEFT models

# Define your Hugging Face username and model IDs
username = "11andrea2233"
finetuned_model_id = f"{username}/gemma-2b-instruct-ft-ai-medical"
base_model_id = "google/gemma-2-2b-it"

# Set the qunatization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    llm_int8_enable_fp32_cpu_offload=True # Enable CPU offloading
)

model_id = "google/gemma-2-2b-it"

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",  # Let accelerate handle device placement
    offload_folder="offload",  # Specify a folder for offloading
    offload_state_dict=True  # Enable offloading of the state dict
)
base_tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

# # Load the Finetuned Model and Tokenizer from local directory
# finetuned_model_path = "merged_model"  # Path where you saved the merged model
# finetuned_model = AutoModelForCausalLM.from_pretrained(
#     finetuned_model_path,
#     torch_dtype=torch.float16,
#     device_map="auto"
# )
# finetuned_tokenizer = AutoTokenizer.from_pretrained(finetuned_model_path)

# Load the Finetuned Model and Tokenizer from HF
finetuned_tokenizer = AutoTokenizer.from_pretrained(model_id)
finetuned_model = AutoModelForCausalLM.from_pretrained(
                  model_id,
                  torch_dtype=torch.float16,
                  device_map="auto"
)

# Ensure the model is in evaluation mode
base_model.eval()
finetuned_model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=

**Perplexity**

In [25]:
test_data

Dataset({
    features: ['Description', 'Patient', 'Doctor', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 25692
})

In [26]:
test_data_subset = test_data.shuffle(seed=42).select(range(10))  # Get 10 random rows

In [57]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from math import exp

def calculate_perplexity(model, tokenizer, dataset):
    total_loss = 0
    total_tokens = 0

    with torch.no_grad():
        for example in dataset:
            inputs = tokenizer(example["prompt"], return_tensors="pt").to("cuda")
            labels = inputs.input_ids
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss
            total_loss += loss.item() * inputs.input_ids.size(1)
            total_tokens += inputs.input_ids.size(1)

    perplexity = exp(total_loss / total_tokens)
    return perplexity

**Semantic Similarity**

In [58]:
import torch
from torch.nn.functional import cosine_similarity

def calculate_semantic_similarity(model, tokenizer, dataset):
    similarities = []
    model.eval()

    with torch.no_grad():
        for example in dataset:
            # Encode the input prompt
            input_ids = tokenizer.encode(example["prompt"], return_tensors="pt").to("cuda")
            # Generate output from the model
            output_ids = model.generate(input_ids, max_length=2500)
            generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

            # Tokenize generated and reference texts
            gen_inputs = tokenizer(generated_text, return_tensors='pt').to("cuda")
            ref_inputs = tokenizer(example["output"], return_tensors='pt').to("cuda")

            # Get embeddings from the last hidden state
            gen_outputs = model(**gen_inputs, output_hidden_states=True, return_dict=True)
            ref_outputs = model(**ref_inputs, output_hidden_states=True, return_dict=True)

            # Average pooling of the embeddings
            gen_embedding = gen_outputs.hidden_states[-1].mean(dim=1).squeeze()
            ref_embedding = ref_outputs.hidden_states[-1].mean(dim=1).squeeze()

            # Compute cosine similarity
            cosine_score = cosine_similarity(gen_embedding, ref_embedding, dim=0).item()
            similarities.append(cosine_score)

    average_similarity = sum(similarities) / len(similarities)
    return average_similarity